# Setup and Imports

In [1]:
import sys
import os
import pandas as pd
from dotenv import load_dotenv

# Thêm đường dẫn thư mục `src` vào Python Path
# Dấu '../' nghĩa là đi lên một cấp (từ `notebooks` ra thư mục gốc) rồi vào `src`
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added '{project_root}' to system path.")

load_dotenv()

Added '/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services' to system path.


True

In [2]:
from src.data.fetchers.yahoo_fetcher import YahooFetchStrategy
from src.data.fetchers.base_fetcher import DataFetchStrategy
from src.data.fecther_factory import create_data_fetcher
from src.data.saver_factory import create_data_saver

## Test fetcher dữ liệu

In [3]:
curr_dir = os.getcwd()
try:
    api_key = os.getenv("API-Key")
    api_secret = os.getenv("Secret-Key")
    data_fetcher = create_data_fetcher("binance", api_key=api_key, api_secret=api_secret)
    btc_data = data_fetcher.fetch_data(
        symbol="BTCUSDT",
        interval="1d", # Lấy khung ngày cho nhanh
        start_str="2025-01-01" # Lấy dữ liệu 1 tháng gần đây
    )
    print("\n--- Fetched BTC Data Sample ---")
    print(btc_data.head())
    assert not btc_data.empty, "Fetched data should not be empty"
    print("\n✅ Test for BinancePriceFetcher PASSED!")
    save_data = create_data_saver("csv")
    path = os.path.join(curr_dir, "..", "data", "raw", "day", "BTCUSDT.csv")
    save_data.save_data(btc_data, file_path=path)
    print("\n✅ Test for CSVSaver PASSED!")

except Exception as e:
    print(f"Error: {e}")



--- Fetched BTC Data Sample ---
                open      high       low     close       volume
timestamp                                                      
2025-01-01  93576.00  95151.15  92888.00  94591.79  10373.32613
2025-01-02  94591.78  97839.50  94392.00  96984.79  21970.48948
2025-01-03  96984.79  98976.91  96100.01  98174.18  15253.82936
2025-01-04  98174.17  98778.43  97514.79  98220.50   8990.05651
2025-01-05  98220.51  98836.85  97276.79  98363.61   8095.63723

✅ Test for BinancePriceFetcher PASSED!

✅ Test for CSVSaver PASSED!


/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/src/utils/standardizer.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit="ms")


In [4]:
try: 
    data_fetcher = create_data_fetcher("yahoo")
    data = data_fetcher.fetch_data(symbol="AAPL", interval="1d", start_date="2025-01-01")
    print("\n--- Fetched AAPL Data Sample ---")
    print(data.head())
    path = os.path.join(curr_dir, "..", "data", "raw", "day", "AAPL.csv")
    save_data.save_data(btc_data, file_path=path)
except Exception as e:
    print(f"Error: {e}")

/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/src/data/fetchers/yahoo_fetcher.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start_date, interval=interval)
[*********************100%***********************]  1 of 1 completed


--- Fetched AAPL Data Sample ---
Price             open        high         low       close      volume
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL
timestamp                                                             
2025-01-02  248.049444  248.218856  240.964609  242.987427  55740700.0
2025-01-03  242.499146  243.316237  241.034344  242.499146  40244100.0
2025-01-06  243.445785  246.455106  242.339711  244.133347  45045600.0
2025-01-07  242.120491  244.681407  240.496267  241.353226  40856000.0
2025-01-08  241.064237  242.847913  239.200857  241.841476  37628900.0


## Test save

In [5]:
try:
    save_data = create_data_saver("csv")
    curr_dir = os.getcwd()
    path = os.path.join(curr_dir, "..", "data", "raw", "day", "btc_data.csv")
    save_data.save_data(btc_data, file_path=path)
    print("\n✅ Test for CSVSaver PASSED!")
except Exception as e:
    print(f"Error: {e}")


✅ Test for CSVSaver PASSED!


In [6]:
from src.data.loader.data_loader_service import DataLoaderService
from src.data.loader.csv_loader import CSVLoader
from src.features.feature_engineer import FeatureEngineer

path = os.path.join(curr_dir, "..", "data", "raw", "day")
repository = CSVLoader(path)
data_loader = DataLoaderService(repository)

raw_data = data_loader.load_data()


# Sử dụng config đúng
config_path = os.path.join(curr_dir, "..", "configs", "main_config.yaml")
feature_engineer = FeatureEngineer(config_path)

data = feature_engineer.process(raw_data, is_training=True)
print("\nProcessed data:")
# print(data.head())

FeatureEngineer initialized with config.
Fitting scaler on training data and transforming...
Feature engineering complete. Final data shape: (588, 23)

Processed data:


/Users/quockhoile/Desktop/DaiHoc/KT/Project/AI-Predict/financial-prediction-system/services/ai-services/src/features/feature_engineer.py:79: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('symbol', group_keys=False).apply(self._create_symbol_features)
